# Distinct entities Algorithm 2

In [ ]:



def check_for_type_1_using_rankings_exp2(segs,
                                         pixelwise_attributions,
                                         segmentwise_attributions,
                                         final_prediction_dictionary,
                                         final_prediction_dictionary_with_percentages,
                                         final_redacted_images,
                                         model_in_use,
                                         model_in_use2,
                                         im,
                                         image,
                                         topk,
                                         top10list,
                                         delta_for_type1,
                                         checkonly=None,
                                         check_both_pred=True):
    combs=[i for i in range(topk)]
    cnt=0
    for (x1,x2) in list(combinations(combs, 2)):
        prediction1=top10list[x1]
        prediction2=top10list[x2]
        first_redaction=final_prediction_dictionary_with_percentages[prediction1]
        second_redaction=final_prediction_dictionary_with_percentages[prediction2]
        print(x1,x2)
        print(id_to_name[prediction1],id_to_name[prediction2])
        print(prediction1,prediction2)
        type1_flag=None
        type2_flag=None

        
        
        if checkonly=='type1' or checkonly==None:
            # checking for type1
            print('checking for type1')
            for i in range(len(first_redaction[prediction1])):
                flag1=None
                if first_redaction[prediction1][i] <= 100-delta_for_type1 and first_redaction[prediction2][i] >= delta_for_type1:
                    flag1=True
                    redaction1_percentile=i
                    break
        
            for i in range(len(first_redaction[prediction1])):
                flag2=None    
                if second_redaction[prediction2][i] <= 100-delta_for_type1 and second_redaction[prediction1][i] >= delta_for_type1:
                    flag2=True
                    redaction2_percentile=i
                    break
        
            if flag1==True and flag2==True:

                mask_id_for_redaction1= list_of_masks[prediction1][:redaction1_percentile]
                mask_id_for_redaction2= list_of_masks[prediction2][:redaction2_percentile]
                mask_id_only1=[i for i in mask_id_for_redaction1 if i not in mask_id_for_redaction2]
                mask_id_only2=[i for i in mask_id_for_redaction2 if i not in mask_id_for_redaction1]
                print(mask_id_for_redaction1)
                print(mask_id_for_redaction2)
                print(mask_id_only1)
                print(mask_id_only2)
                
                mask_only1=get_mask(mask_id_only1,new_segs)
                masking_type='normal/'
                new_im=mask_on_image(im , mask_only1, masking_type)
                new_im1=mask_on_image(image , mask_only1, masking_type)
                (predictions3,decoded_predictions3,top10list3,top1000list3,id_to_name3)=predict_img(model_in_use,new_im,print_data=False)
                new_prediction1_percentage_after_only1=(predictions3[0][prediction1].numpy()/predictions[0][prediction1].numpy())*100
                new_prediction2_percentage_after_only1=(predictions3[0][prediction2].numpy()/predictions[0][prediction2].numpy())*100
                plt.imshow(mask_only1)
                plt.show()
                plt.imshow(new_im1)
                plt.show()
                print('redaction of only1 of '+str(prediction1)+' and '+str(prediction2))
                print('redaction of only of '+id_to_name[prediction1]+' and '+id_to_name[prediction2])
                print(' Initial predcition of '+str(prediction1),final_prediction_dictionary[prediction1][prediction1][0])
                print(' final predcition of '+str(prediction1),predictions3[0][prediction1].numpy())
                print(' final percentage of '+str(prediction1),new_prediction1_percentage_after_only1)
                print(' Initial predcition of '+str(prediction2),final_prediction_dictionary[prediction2][prediction2][0])
                print(' final predcition of '+str(prediction2),predictions3[0][prediction2].numpy())
                print(' final percentage of '+str(prediction2),new_prediction2_percentage_after_only1)

                
                mask_only2=get_mask(mask_id_only2,new_segs)
                masking_type='normal/'
                new_im=mask_on_image(im , mask_only2, masking_type)
                new_im1=mask_on_image(image , mask_only2, masking_type)
                (predictions3,decoded_predictions3,top10list3,top1000list3,id_to_name3)=predict_img(model_in_use,new_im,print_data=False)
                new_prediction1_percentage_after_only2=(predictions3[0][prediction1].numpy()/predictions[0][prediction1].numpy())*100
                new_prediction2_percentage_after_only2=(predictions3[0][prediction2].numpy()/predictions[0][prediction2].numpy())*100
                plt.imshow(mask_only2)
                plt.show()
                plt.imshow(new_im1)
                plt.show()
                print('redaction of intersection of '+str(prediction1)+' and '+str(prediction2))
                print('redaction of intersection of '+id_to_name[prediction1]+' and '+id_to_name[prediction2])
                print(' Initial predcition of '+str(prediction1),final_prediction_dictionary[prediction1][prediction1][0])
                print(' final predcition of '+str(prediction1),predictions3[0][prediction1].numpy())
                print(' final percentage of '+str(prediction1),new_prediction1_percentage_after_only2)
                print(' Initial predcition of '+str(prediction2),final_prediction_dictionary[prediction2][prediction2][0])
                print(' final predcition of '+str(prediction2),predictions3[0][prediction2].numpy())
                print(' final percentage of '+str(prediction2),new_prediction2_percentage_after_only2)

                if new_prediction1_percentage_after_only1 <= 100-delta_for_type1 and new_prediction2_percentage_after_only1 >= delta_for_type1 and new_prediction2_percentage_after_only2 <= 100-delta_for_type1 and new_prediction1_percentage_after_only2 >= delta_for_type1:
                    type1_flag=True
                    # cnt=cnt+1
                    print('Yes its type1')
                    
                    
                    print('redaction for '+str(prediction1)+' percentile at ',redaction1_percentile)
                    print(' Initial predcition of '+str(prediction1),final_prediction_dictionary[prediction1][prediction1][0])
                    print(' final predcition of '+str(prediction1),final_prediction_dictionary[prediction1][prediction1][redaction1_percentile])
                    print(' final percentage of '+str(prediction1),first_redaction[prediction1][redaction1_percentile])
                    print(' Initial predcition of '+str(prediction2),final_prediction_dictionary[prediction1][prediction2][0])
                    print(' final predcition of '+str(prediction2),final_prediction_dictionary[prediction1][prediction2][redaction1_percentile])
                    print(' final percentage of '+str(prediction2),first_redaction[prediction2][redaction1_percentile])
        
        
                    redacted_mask_with_pred1=get_mask(mask_id_for_redaction1,new_segs)
                    redacted_image_with_pred1=mask_on_image(image , redacted_mask_with_pred1, masking_type)
                    plot_for_type1_redaction1(redacted_image_with_pred1,redacted_mask_with_pred1,final_prediction_dictionary,first_redaction,id_to_name,prediction1,prediction2,redaction1_percentile,'prob')
        
                    # print(len(first_redaction[prediction1]))
                    
                    print('redaction for '+str(prediction2)+' percentile at ',redaction2_percentile)
                    print(' Initial predcition of '+str(prediction2),final_prediction_dictionary[prediction2][prediction2][0])
                    print(' final predcition of '+str(prediction2),final_prediction_dictionary[prediction2][prediction2][redaction2_percentile])
                    print(' final percentage of '+str(prediction2),second_redaction[prediction2][redaction2_percentile])
                    print(' Initial predcition of '+str(prediction1),final_prediction_dictionary[prediction2][prediction1][0])
                    print(' final predcition of '+str(prediction1),final_prediction_dictionary[prediction2][prediction1][redaction2_percentile])
                    print(' final percentage of '+str(prediction1),second_redaction[prediction1][redaction2_percentile])
        
                    redacted_mask_with_pred2=get_mask(mask_id_for_redaction2,new_segs)
                    redacted_image_with_pred2=mask_on_image(image , redacted_mask_with_pred2, masking_type)
                    plot_for_type1_redaction2(redacted_image_with_pred2,redacted_mask_with_pred2,final_prediction_dictionary,second_redaction,id_to_name,prediction1,prediction2,redaction2_percentile)
        
                    
                
                
                # print(id_to_name[prediction1])
                # plt.imshow(redacted_image_with_pred1)
                # plt.show()
                # print(id_to_name[prediction2])
                # plt.imshow(redacted_image_with_pred2)
                # plt.show()


        
        # break
    if type1_flag==True and type2_flag==True:
        print('Here i am , Both are true')
    # print(cnt)
    return [],[],[],[]
    # return mask_id_for_redaction1,mask_id_for_redaction2,mask_id_only1,mask_id_only2
    # return intersection_predictions,new_prediction1_percentage_after_intersection,new_prediction2_percentage_after_intersection
    
    
    


